<a href="https://colab.research.google.com/github/busranur-sr/ClimateChange/blob/main/co2regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

%matplotlib inline


In [131]:
df = pd.read_csv('out.csv')

df['Date']=df['Date'].str.split(expand=True)[0] # Date verisi object halindeydi. İçinde veri "1990 [YR1990]" bu şekilde bulunuyordu. İlk kısım çekildi.	
df=df.replace('..', np.nan) # Boş veriler .. şeklinde gösteriliyordu. Nan haline getirildi.
df=df.rename(columns={"CO2 emissions from liquid fuel consumption (kt)": "Liquid Co",  # Column isimlerinin düzenlenmesi
                      "CO2 emissions from gaseous fuel consumption (kt)" :"Gaseous Co",
                      "CO2 emissions from solid fuel consumption (kt)" : "Solid Co",
                      "CO2 emissions (kt)":"Total Co"})
s = df.drop('Country Name',1).select_dtypes(include='object').columns  # data type object-> float. Üzerinde işlem yapabilmek için 
df[s] = df[s].astype("float")
df=df.astype({'Date':'int16','Country Name':'str'})
df=df[df['Date']<2016] # Rest of years's data is empty


In [132]:
nan_count = df.isnull().sum()
drop_column = nan_count[nan_count<600]
display(drop_column)
df = df[drop_column.index]
df =df.dropna()
display(df)

Country Name                                                                                   0
Date                                                                                           0
Access to electricity (% of population)                                                      196
Agricultural land (% of land area)                                                            97
Agricultural land (sq. km)                                                                    91
Agriculture, forestry, and fishing, value added (% of GDP)                                   256
Arable land (% of land area)                                                                 116
CO2 emissions (kg per 2015 US$ of GDP)                                                       264
CO2 emissions (kg per 2017 PPP $ of GDP)                                                     294
CO2 emissions (kg per PPP $ of GDP)                                                          277
Total Co                      

,Country Name,Date,Access to electricity (% of population),Agricultural land (% of land area),Agricultural land (sq. km),"Agriculture, forestry, and fishing, value added (% of GDP)",Arable land (% of land area),CO2 emissions (kg per 2015 US$ of GDP),CO2 emissions (kg per 2017 PPP $ of GDP),CO2 emissions (kg per PPP $ of GDP),Total Co,CO2 emissions (metric tons per capita),CO2 emissions from gaseous fuel consumption (% of total),Gaseous Co,CO2 emissions from liquid fuel consumption (% of total),Liquid Co,CO2 emissions from solid fuel consumption (% of total),Solid Co,Cereal yield (kg per hectare),Electricity production from coal sources (% of total),Electricity production from hydroelectric sources (% of total),Electricity production from natural gas sources (% of total),Electricity production from oil sources (% of total),"Electricity production from renewable sources, excluding hydroelectric (% of total)","Electricity production from renewable sources, excluding hydroelectric (kWh)","Foreign direct investment, net inflows (% of GDP)",Forest area (% of land area),Forest area (sq. km),Methane emissions (kt of CO2 equivalent),"Mortality rate, under-5 (per 1,000 live births)",Nitrous oxide emissions (thousand metric tons of CO2 equivalent),"Other greenhouse gas emissions, HFC, PFC and SF6 (thousand metric tons of CO2 equivalent)",Population growth (annual %),"Population, total",Renewable electricity output (% of total electricity output),Renewable energy consumption (% of total final energy consumption),Total greenhouse gas emissions (kt of CO2 equivalent),Urban population,Urban population (% of total population),Urban population growth (annual %)
37,Albania,2000,100.000000,41.751825,11440.0,24.515361,21.094891,0.523296,0.174137,0.265687,3170.0,1.026213,0.694069,22.002,89.766309,2845.592,2.197886,69.673,3175.1,0.000000,96.149016,0.0,3.850984,0.000000,0.0,4.108776,28.076642,7693.000,3750.0,27.2,1180.0,150.656036,-0.637357,3089027.0,96.149016,41.445000,8220.0,1289391.0,41.741,0.742479
38,Albania,2011,100.000000,43.832117,12010.0,18.226765,22.700730,0.471671,0.156958,0.169952,5040.0,1.734823,0.582063,29.336,74.940675,3777.010,10.622659,535.382,4750.7,0.000000,98.592221,0.0,1.407779,0.000000,0.0,8.135333,28.594653,7834.935,3120.0,12.1,1170.0,1014.183540,-0.269017,2905195.0,98.592221,35.961102,9630.0,1546929.0,53.247,1.787784
39,Albania,2012,99.900000,43.843066,12013.0,18.766799,22.594891,0.422631,0.140639,0.150014,4580.0,1.579092,0.640524,29.336,68.936397,3157.287,13.691201,627.057,4883.8,0.000000,100.000000,0.0,0.000000,0.000000,0.0,7.451357,28.646606,7849.170,3170.0,11.2,1180.0,681.643684,-0.165151,2900401.0,100.000000,40.046700,9240.0,1575788.0,54.330,1.848379
40,Albania,2013,100.000000,43.332117,11873.0,19.565176,22.521898,0.437624,0.145628,0.156516,4790.0,1.654524,0.688998,33.003,69.588789,3333.303,11.483299,550.050,4949.8,0.000000,100.000000,0.0,0.000000,0.000000,0.0,9.816290,28.698558,7863.405,3190.0,10.4,1150.0,715.218840,-0.183211,2895092.0,100.000000,41.286999,9440.0,1603505.0,55.387,1.743639
41,Albania,2014,99.950000,42.857299,11742.9,19.990154,22.467153,0.468595,0.155934,0.160471,5220.0,1.806789,1.123985,58.672,73.972241,3861.351,13.417567,700.397,4892.6,0.000000,100.000000,0.0,0.000000,0.000000,0.0,8.693041,28.750511,7877.640,3230.0,9.9,1170.0,709.886387,-0.207047,2889104.0,100.000000,38.697399,9930.0,1630119.0,56.423,1.646116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3182,Zimbabwe,2011,36.900000,42.393693,164000.0,8.665865,10.856921,0.713597,0.348573,0.423960,11490.0,0.891090,0.000000,0.000,28.404091,3263.630,50.999704,5859.866,587.7,41.655816,56.434462,0.0,0.488281,1.421441,131000000.0,2.441511,46.165723,178592.100,11820.0,81.4,6860.0,1469.966370,1.536406,12894323.0,57.855903,79.509903,30550.0,4257061.0,33.015,0.989668
3183,Zimbabwe,2012,44.000000,41.876696,162000.0,8.044518,10.339925,0.644667,0.314

In [133]:
#drop output values in input
df = df.drop(['Total greenhouse gas emissions (kt of CO2 equivalent)','Solid Co','Liquid Co','Gaseous Co','CO2 emissions (kg per 2015 US$ of GDP)',
              'CO2 emissions (kg per 2017 PPP $ of GDP)','CO2 emissions (kg per PPP $ of GDP)'],axis = 1)
df

,Country Name,Date,Access to electricity (% of population),Agricultural land (% of land area),Agricultural land (sq. km),"Agriculture, forestry, and fishing, value added (% of GDP)",Arable land (% of land area),Total Co,CO2 emissions (metric tons per capita),CO2 emissions from gaseous fuel consumption (% of total),CO2 emissions from liquid fuel consumption (% of total),CO2 emissions from solid fuel consumption (% of total),Cereal yield (kg per hectare),Electricity production from coal sources (% of total),Electricity production from hydroelectric sources (% of total),Electricity production from natural gas sources (% of total),Electricity production from oil sources (% of total),"Electricity production from renewable sources, excluding hydroelectric (% of total)","Electricity production from renewable sources, excluding hydroelectric (kWh)","Foreign direct investment, net inflows (% of GDP)",Forest area (% of land area),Forest area (sq. km),Methane emissions (kt of CO2 equivalent),"Mortality rate, under-5 (per 1,000 live births)",Nitrous oxide emissions (thousand metric tons of CO2 equivalent),"Other greenhouse gas emissions, HFC, PFC and SF6 (thousand metric tons of CO2 equivalent)",Population growth (annual %),"Population, total",Renewable electricity output (% of total electricity output),Renewable energy consumption (% of total final energy consumption),Urban population,Urban population (% of total population),Urban population growth (annual %)
37,Albania,2000,100.000000,41.751825,11440.0,24.515361,21.094891,3170.0,1.026213,0.694069,89.766309,2.197886,3175.1,0.000000,96.149016,0.0,3.850984,0.000000,0.0,4.108776,28.076642,7693.000,3750.0,27.2,1180.0,150.656036,-0.637357,3089027.0,96.149016,41.445000,1289391.0,41.741,0.742479
38,Albania,2011,100.000000,43.832117,12010.0,18.226765,22.700730,5040.0,1.734823,0.582063,74.940675,10.622659,4750.7,0.000000,98.592221,0.0,1.407779,0.000000,0.0,8.135333,28.594653,7834.935,3120.0,12.1,1170.0,1014.183540,-0.269017,2905195.0,98.592221,35.961102,1546929.0,53.247,1.787784
39,Albania,2012,99.900000,43.843066,12013.0,18.766799,22.594891,4580.0,1.579092,0.640524,68.936397,13.691201,4883.8,0.000000,100.000000,0.0,0.000000,0.000000,0.0,7.451357,28.646606,7849.170,3170.0,11.2,1180.0,681.643684,-0.165151,2900401.0,100.000000,40.046700,1575788.0,54.330,1.848379
40,Albania,2013,100.000000,43.332117,11873.0,19.565176,22.521898,4790.0,1.654524,0.688998,69.588789,11.483299,4949.8,0.000000,100.000000,0.0,0.000000,0.000000,0.0,9.816290,28.698558,7863.405,3190.0,10.4,1150.0,715.218840,-0.183211,2895092.0,100.000000,41.286999,1603505.0,55.387,1.743639
41,Albania,2014,99.950000,42.857299,11742.9,19.990154,22.467153,5220.0,1.806789,1.123985,73.972241,13.417567,4892.6,0.000000,100.000000,0.0,0.000000,0.000000,0.0,8.693041,28.750511,7877.640,3230.0,9.9,1170.0,709.886387,-0.207047,2889104.0,100.000000,38.697399,1630119.0,56.423,1.646116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3182,Zimbabwe,2011,36.900000,42.393693,164000.0,8.665865,10.856921,11490.0,0.891090,0.000000,28.404091,50.999704,587.7,41.655816,56.434462,0.0,0.488281,1.421441,131000000.0,2.441511,46.165723,178592.100,11820.0,81.4,6860.0,1469.966370,1.536406,12894323.0,57.855903,79.509903,4257061.0,33.015,0.989668
3183,Zimbabwe,2012,44.000000,41.876696,162000.0,8.044518,10.339925,12110.0,0.923360,0.000000,30.523006,28.403353,696.4,39.052483,58.567858,0.0,0.554167,1.825492,168000000.0,2.044131,46.046633,178131.400,11730.0,73.8,6510.0,1827.732117,1.698084,13115149.0,60.393350,77.822197,4306228.0,32.834,1.148333
3184,Zimbabwe,2013,38.336353,41.876696,162000.0,7.144479,10.339925,12360.0,0.925816,0.000000,33.287816,57.170785,669.3,45.464995,52.204807,0.0,0.595611,1.734587,166000000.0,1.954060,45.927543,177670.700,11700.0,67.8,6310.0,1972.408020,1.777672,13350378.0,53.939394,79.305901,4359432.0,32.654,1.227943
3185,Zimbabwe,2014,32.300000,41.876696,162000.0,8.74530

In [134]:
X = df.drop('Total Co', axis = 1)
X = X.drop('Country Name', axis = 1)

Y = df[['Total Co']]

# Split X and y into X_
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=1)
display(X_train)
display
# create a Linear Regression model object
regression_model = LinearRegression()

# pass through the X_train & y_train data set
regression_model.fit(X_train, y_train)


,Date,Access to electricity (% of population),Agricultural land (% of land area),Agricultural land (sq. km),"Agriculture, forestry, and fishing, value added (% of GDP)",Arable land (% of land area),CO2 emissions (metric tons per capita),CO2 emissions from gaseous fuel consumption (% of total),CO2 emissions from liquid fuel consumption (% of total),CO2 emissions from solid fuel consumption (% of total),Cereal yield (kg per hectare),Electricity production from coal sources (% of total),Electricity production from hydroelectric sources (% of total),Electricity production from natural gas sources (% of total),Electricity production from oil sources (% of total),"Electricity production from renewable sources, excluding hydroelectric (% of total)","Electricity production from renewable sources, excluding hydroelectric (kWh)","Foreign direct investment, net inflows (% of GDP)",Forest area (% of land area),Forest area (sq. km),Methane emissions (kt of CO2 equivalent),"Mortality rate, under-5 (per 1,000 live births)",Nitrous oxide emissions (thousand metric tons of CO2 equivalent),"Other greenhouse gas emissions, HFC, PFC and SF6 (thousand metric tons of CO2 equivalent)",Population growth (annual %),"Population, total",Renewable electricity output (% of total electricity output),Renewable energy consumption (% of total final energy consumption),Urban population,Urban population (% of total population),Urban population growth (annual %)
893,2014,100.000000,43.398472,1.162669e+06,1.578544,24.018953,6.523712,24.851725,45.038884,26.668261,6100.111461,21.797114,11.989083,14.804089,2.362455,17.249322,3.863410e+11,2.844478,38.307313,1.026274e+06,291720.0,3.880677,161910.0,-12017.882656,0.343942,3.384622e+08,29.242953,16.248797,2.576649e+08,76.128099,0.606901
2114,2011,78.712410,42.138109,5.070900e+04,18.884220,12.489613,0.808059,0.000000,95.095996,0.000000,1994.800000,0.000000,11.610879,0.000000,66.004184,22.384937,8.560000e+08,9.579152,34.336264,4.132026e+04,8370.0,22.900000,2950.0,251.622192,1.346936,5.903035e+06,33.995816,53.299702,3.371400e+06,57.113000,1.690697
2943,2012,100.000000,49.903200,3.840700e+05,7.694878,26.736224,4.405565,25.949310,23.397083,40.527887,2958.300000,28.398387,24.161155,43.632879,0.684354,3.069362,7.351000e+09,1.560831,27.678193,2.130197e+05,46610.0,15.800000,29590.0,-2793.738281,1.631148,7.465105e+07,27.230517,13.029000,5.372934e+07,71.974000,2.429055
2332,2013,87.500000,41.721166,1.244000e+05,12.473425,18.747694,0.989162,6.895310,42.357974,40.959415,3531.800000,42.623495,13.311455,24.966120,5.966838,13.092233,9.854000e+09,1.316427,23.289996,6.944378e+04,65610.0,30.600000,12470.0,5153.800781,1.692088,9.887156e+07,26.403688,27.838900,4.538501e+07,45.903000,2.109049
771,2012,96.756383,48.960598,7.750419e+06,9.898506,12.059399,5.461048,4.965102,15.757415,73.495976,5180.739965,70.396356,16.888902,6.742515,0.964317,2.782048,1.589170e+11,2.845703,30.024506,4.752852e+06,1807700.0,20.681114,697510.0,-179043.030296,0.727625,1.969561e+09,19.805665,14.649544,9.756245e+08,49.535114,2.854558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499,2012,56.500000,46.564172,8.965000e+04,14.097692,17.140186,0.556634,0.737332,63.017346,10.961676,1221.300000,0.000000,8.405797,4.057971,82.666667,1.420290,4.900000e+07,1.563680,43.568067,8.388160e+04,9790.0,60.000000,7050.0,947.733978,2.785615,1.340199e+07,9.826087,50.373699,5.977690e+06,44.603000,3.720403
314,2011,88.335943,34.669067,3.755700e+05,9.769586,4.004431,1.610700,37.249587,50.736122,0.000000,2045.300000,0.000000,32.506925,64.210526,1.592798,1.689751,1.220000e+08,3.584442,48.808504,5.287425e+05,22710.0,38.900000,8460.0,208.361328,1.622360,1.021295e+07,34.196676,13.806300,6.829809e+06,66.874000,2.288511
3052,2013,99.090197,35.509295,1.902163e+07,7.099586,9.129493,6.310044,13.617184,21.063787,59.263972,4180.010060,53.620831,20.511117,15.548454,2.558735,3.228594,2.939170e+11,3.025233,37.495190,2.008544e

LinearRegression()

In [135]:
y_prediction =  regression_model.predict(X_test)
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score = r2_score(y_test,y_prediction)
score  

0.9801777069266452